This notebook provides basic examples of how to use my inference code

In [1]:
import sys
sys.path.insert(0,'..')
from gp3.inference.svi import GPSVI
from gp3.inference.laplace import GPLaplace
from gp3.likelihoods.likelihoods import PoissonLike
from gp3.utils import data_utils as sim
from gp3.utils.grid_utils import fill_grid
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import GPy
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
import numpy as np

First, let's simulate some data on an equispaced grid. We will do this in 2D for sake of visualization.

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, k=GPy.kern.RBF(2, variance=1.0, lengthscale=40.), mu = 3.)
y = sim.poisson_draw(f, 1.)

In [3]:
trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,))
trace_draws = go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



Now, we run inference using both the SVI method and the Laplace method.

In [4]:
mu = np.ones([X.shape[0]])*np.log(np.mean(y))
inf_svi = GPSVI(GPy.kern.RBF(1, 1., 40.), PoissonLike(), X, y, mu, verbose = True)
inf_lp = GPLaplace(mu, GPy.kern.RBF(1, 1., 40.), PoissonLike(), X, y)
inf_svi.run(500)
inf_lp.run(10)

ELBO: 14900.92 | KL: 2192.67 | logL: 17093.60 | step: 0.0001220703125: 100%|██████████| 500/500 [00:08<00:00, 57.95it/s] 


Here, we make predictions and plot the inferred functions.

In [5]:
pred_svi = inf_svi.predict()
pred_lp = inf_lp.f_pred

trace_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_svi, mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior mean')
trace_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_lp, mode = 'markers', marker=dict(size = 2,), name = 'Laplace posterior mean')
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_svi, 1, 1)
fig.append_trace(trace_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



To get the predictive variances, we do the following. For the SVI method, we already have them calculated. We use covariance estimation with Gaussian perturbations for the Laplace method, where the parameter indicates the number of samples to estimate based on.

In [6]:
svi_variances = np.diag(inf_svi.full_S())
lp_variances = inf_lp.variance(500)

var_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=svi_variances, mode = 'markers', marker=dict(size = 2,), name = 'SVI posterior variances')
var_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=lp_variances, mode = 'markers', marker=dict(size = 2,), name = 'Laplace posterior variances')
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(var_svi, 1, 1)
fig.append_trace(var_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



For SVI, we can look at the valeus of the variational objective, likelihood, and KL terms as a function of iteration.

In [7]:
iplot([go.Scatter(x = range(len(inf_svi.elbos)), y = inf_svi.elbos)])
iplot([go.Scatter(x = range(len(inf_svi.likes)), y = inf_svi.likes)])
iplot([go.Scatter(x = range(len(inf_svi.kls)), y = inf_svi.kls)])

## Partial Grids

Here, we take a sample of 20 percent of the above grid to "observe"

In [8]:
X_part, y_part = sim.rand_partial_grid(X, y, 0.2)
X_full, y_full, obs_idx, imag_idx = fill_grid(X_part, y_part)

color = np.zeros(X_full.shape[0])
color[obs_idx] = 1.0
trace_partial_obs = go.Scatter3d(x = X_full[obs_idx, 0], y = X_full[obs_idx, 1],
                                 z= y[obs_idx], mode = 'markers', marker=dict(size = 2))
iplot([trace_partial_obs])

We can run inference on partial grids by passing in the locations of the full grid, the indices of the observed points, and the values of y at the observed locations.

In [9]:
inf_svi = GPSVI(GPy.kern.RBF(1, 3., 40.), PoissonLike(), X, y_part, mu, obs_idx = obs_idx, noise = 1e-2, verbose = True)
inf_lp = GPLaplace(mu, GPy.kern.RBF(1, 1., 40.), PoissonLike(), X, y_part, obs_idx=obs_idx)
inf_svi.run(500)
inf_lp.run(10)

ELBO: 779.78 | KL: 2147.99 | logL: 2927.77 | step: 0.0001220703125: 100%|██████████| 500/500 [00:08<00:00, 58.03it/s]          


We can make predictions of the entire function below.

In [10]:
pred_svi = inf_svi.predict()
pred_lp = inf_lp.f_pred

trace_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_svi, mode = 'markers', marker=dict(size = 2,), name = "SVI partial grid posterior mean")
trace_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=pred_lp, mode = 'markers', marker=dict(size = 2,), name = "Laplace partial grid posterior mean")
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_svi, 1, 1)
fig.append_trace(trace_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



We can get predictive variances in the same way as the full grid case.

In [11]:
svi_variances = np.diag(inf_svi.full_S())
lp_variances = inf_lp.variance(50)

var_svi = go.Scatter3d(x = X[:,0], y = X[:,1], z=svi_variances, mode = 'markers', marker=dict(size = 2,))
var_lp = go.Scatter3d(x = X[:,0], y = X[:,1], z=lp_variances, mode = 'markers', marker=dict(size = 2,))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(var_svi, 1, 1)
fig.append_trace(var_lp, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]

